In [ ]:
!pip install -qU langchain-pinecone pinecone

### Credentials
Create a new Pinecone account, or sign into your existing one, and create an API key to use in this notebook.

In [1]:
import getpass
import os

from pinecone import Pinecone

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

/Users/jamesbriggs/Documents/pointman/pinecone/aurelio-langchain-pinecone/langchain-pinecone/libs/pinecone/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialization
Before initializing our vector store, let's connect to a Pinecone index. If one named index_name doesn't exist, it will be created.

In [2]:
index_name = "langchain-sparse-vector-search"  # change if desired

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"pinecone-sparse-english-v0",
            "field_map":{"text": "chunk_text"},
            "metric": "dotproduct",
            "read_parameters": {},
            "write_parameters": {}
        }
    )

index = pc.Index(index_name)
print(f"Index `{index_name}` host: {index.config.host}")

Index `langchain-sparse-vector-search` host: https://langchain-sparse-vector-search-yrrgefy.svc.aped-4627-b74a.pinecone.io


For our sparse embedding model we use [`pinecone-sparse-english-v0`](https://docs.pinecone.io/models/pinecone-sparse-english-v0), we initialize it like so:

In [3]:
from langchain_pinecone import PineconeSparseEmbeddings

sparse_embeddings = PineconeSparseEmbeddings(model="pinecone-sparse-english-v0")

Now that our Pinecone index and embedding model are both ready, we can initialize our sparse vector store in LangChain:

In [4]:
from langchain_pinecone import PineconeSparseVectorStore

vector_store = PineconeSparseVectorStore(index=index, embedding=sparse_embeddings)

### Manage vector store
Once you have created your vector store, we can interact with it by adding and deleting different items.

#### Add items to vector store
We can add items to our vector store by using the `add_documents` function.

In [10]:
from uuid import uuid4

from langchain_core.documents import Document

documents = [
    Document(
        page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
        metadata={"source": "social"},
    ),
    Document(
        page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
        metadata={"source": "news"},
    ),
    Document(
        page_content="Building an exciting new project with LangChain - come check it out!",
        metadata={"source": "social"},
    ),
    Document(
        page_content="Robbers broke into the city bank and stole $1 million in cash.",
        metadata={"source": "news"},
    ),
    Document(
        page_content="Wow! That was an amazing movie. I can't wait to see it again.",
        metadata={"source": "social"},
    ),
    Document(
        page_content="Is the new iPhone worth the price? Read this review to find out.",
        metadata={"source": "website"},
    ),
    Document(
        page_content="The top 10 soccer players in the world right now.",
        metadata={"source": "website"},
    ),
    Document(
        page_content="LangGraph is the best framework for building stateful, agentic applications!",
        metadata={"source": "social"},
    ),
    Document(
        page_content="The stock market is down 500 points today due to fears of a recession.",
        metadata={"source": "news"},
    ),
    Document(
        page_content="I have a bad feeling I am going to get deleted :(",
        metadata={"source": "social"},
    ),
]

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['8c6485d5-4d7c-4953-a0b3-8661116038f5',
 '168b57fe-f566-4f9e-b76e-5186bee2f201',
 '404fa844-4e47-4c25-a071-4807e5e2c849',
 '89678148-b650-4307-9b46-d5ee3be96132',
 '95463694-4403-4900-a541-839b6212461f',
 '8e7be680-b175-485e-866b-0eb7f59f42f0',
 'b47e15b9-9e37-4402-aa24-94cc05bbc53e',
 '606e0db6-73dd-4805-844a-4cee759c9ec5',
 '1ac1723f-1e9f-473f-87af-d0df94b03e52',
 '21bbafe3-ecc9-4e55-9ffc-0ed45f48eff1']

#### Delete items from vector store

We can delete records from our vector store using the `delete` method, providing it with a list of document IDs to delete.

In [6]:
vector_store.delete(ids=[uuids[-1]])

## Querying the Vector Store

Once we have loaded our documents into the vector store we're most likely ready to begin querying. There are various method for doing this in LangChain.

First, we'll see how to perform a simple vector search by querying our `vector_store` directly via the `similarity_search` method:

In [8]:
results = vector_store.similarity_search("I'm building a new LangChain project!", k=3)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Is the new iPhone worth the price? Read this review to find out. [{'source': 'website'}]


We can also add metadata filtering to our query to limit our search based on various criteria. Let's try limiting the sources to only include tweets:

In [9]:
results = vector_store.similarity_search(
    "I'm building a new LangChain project!",
    k=3,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]


When comparing these results, we can see that our first query returned t

##### Similarity search with score

We can also search and return both the response and the similarity as a list of tuples.

In [21]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=3, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=4.640625] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


#### Query by turning into retriever
You can also transform the vector store into a retriever for easier usage in your chains.

In [22]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

/Users/jakit/customers/aurelio/langchain-pinecone/libs/pinecone/.venv/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1077: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='38587162-422c-41aa-bad9-8dd6c8025d90', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'), 2.3984375)]
  self.vectorstore.similarity_search_with_relevance_scores(


[Document(id='38587162-422c-41aa-bad9-8dd6c8025d90', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]